## **Notebook Content**

Project: *Seller Fulfillment Intervention Monitor*

File: manual_data_cleanup.ipynb <br>
Author: Bryan Melvida

Purpose:
- Manually inspect raw dataset issues surfaced by CI validation
- Identify root causes of structural and semantic failures
- Define deterministic cleaning rules for downstream automation

Input: [`data/raw/train/`](../data/raw/train/) <br>

---

CI Failure Summary:

> [**ERROR**] df_Orders: 9 unparsable timestamp value(s) in `order_approved_at` <br>
> [**ERROR**] df_Orders: 1889 unparsable timestamp value(s) in `order_delivered_timestamp` <br>
> [**ERROR**] df_products: 61865 duplicated primary key value(s) 

Full CI output: [`raw_ci_pre_cleaning.log`](../logs/raw_ci_pre_cleaning.log)

<br>

---

In [9]:
import pandas as pd

df_orders_train = pd.read_csv('../data/raw/train/df_Orders.csv')
df_products_train = pd.read_csv('../data/raw/train/df_Products.csv')

<br>

---

## **Investigation**

### **Unparsable Timestamps**

In [10]:
order_delivered_mask = pd.to_datetime(df_orders_train['order_delivered_timestamp'], errors= 'coerce').isna()
order_approved_mask = pd.to_datetime(df_orders_train['order_approved_at'], errors= 'coerce').isna()

print(f'`order_delivered_timestamps` Unparsable timestamps values: {order_delivered_mask.sum()}')
print(f'`order_approved_at` Unparsable timestamps values: {order_approved_mask.sum()}')

`order_delivered_timestamps` Unparsable timestamps values: 1889
`order_approved_at` Unparsable timestamps values: 9


In [11]:
df_orders_train.loc[order_delivered_mask].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
15,P5R6jr1qZdh4,FrEvnEiMKGpr,canceled,2017-07-24 11:38:43,2017-07-24 11:50:18,NaN,2017-08-07
41,C21fWds5zL0W,iFsAJRrzVaTS,shipped,2017-02-04 12:58:55,2017-02-04 13:10:38,NaN,2017-03-15
86,vSug5AqRo9gE,oWBBqGwqpx4m,shipped,2017-05-07 10:22:58,2017-05-09 09:35:20,NaN,2017-06-12
89,2HQ26ShSPhCA,uvuFFZDOAlU7,canceled,2017-07-29 12:56:17,2017-07-29 13:05:18,NaN,2017-08-18
133,1By8LOosrvF2,grsbZ5L1H5ty,canceled,2017-11-06 15:47:20,2017-11-07 07:30:29,NaN,2017-11-28


In [12]:
df_orders_train.loc[order_approved_mask].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
10788,4Itts2TBUQzs,Z3dI7uRBfJOD,delivered,2017-02-17 13:05:55,NaN,2017-03-02 11:09:19,2017-03-20
16137,6MPB830Ufh0n,SOtqXN1MqcLj,delivered,2017-02-17 17:21:55,NaN,2017-03-03 12:16:03,2017-03-20
28951,lqW95yj4n1Qd,jCPmhkcyVR1z,delivered,2017-02-18 15:52:27,NaN,2017-03-07 13:57:47,2017-03-29
30494,dn1i9D4DFVZ2,Es95IGZCDDXy,delivered,2017-02-17 15:53:27,NaN,2017-03-03 11:47:47,2017-03-23
41752,vBZCEuCG9Hf1,zHEH7vdeO6UZ,delivered,2017-02-18 11:04:19,NaN,2017-03-02 16:15:23,2017-03-22


<br>

---

### **Duplicated Primary Keys**

In [13]:
product_duplicates = df_products_train.duplicated(subset= 'product_id')

print(f'Total product duplicated primary keys: {product_duplicates.sum()}')

Total product duplicated primary keys: 61865


In [14]:
df_products_train.loc[product_duplicates].sort_values(by= 'product_id')

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
49062,00ehq6lzh6MX,auto,100.0,21.0,2.0,17.0
54062,00ehq6lzh6MX,auto,100.0,21.0,2.0,17.0
33688,00ehq6lzh6MX,auto,100.0,21.0,2.0,17.0
69149,00ehq6lzh6MX,auto,100.0,21.0,2.0,17.0
31373,00ehq6lzh6MX,auto,100.0,21.0,2.0,17.0
...,...,...,...,...,...,...
47690,zzVsv7FKKs9m,housewares,1000.0,30.0,5.0,35.0
75391,zzddahYAwwwB,toys,1400.0,40.0,30.0,25.0
51720,zztEAImyIjb9,toys,300.0,35.0,3.0,12.0
61802,zztEAImyIjb9,toys,300.0,35.0,3.0,12.0


<br>

---

## **Automation Plan**

>**Enforce non-negotiable field completeness**
>
>- Rows missing required timestamps or seller identifiers cannot support valid aggregation and will be excluded.
>
>**Enforce valid temporal ordering**
>
>- Records with impossible or unparsable time relationships will be excluded to preserve event integrity.
>
>**Remove exact duplicate events**
>
>- Identical records representing the same event will be deduplicated to prevent double counting.
>
>**Reject primary key violations**
>
>- Declared primary key uniqueness will be enforced; violations will fail contract enforcement rather than being corrected.

### **Explicit Non-Automation Boundaries**

- No imputation of missing identifiers or timestamps
- No inferred corrections or business logic
- No thresholding, scoring, or eligibility rules

*Any violations outside this scope remain CI-blocking and <ins>require explicit human review</ins>*.




---

## **End of Notebook**